<h1>Predicting Stock Movement Using Random Forest</h1>
Repo: https://github.com/areed1192/sigma_coding_youtube/blob/master/python/python-data-science/machine-learning/random-forest/random_forest_price_prediction.ipynb


<h2>Random Forest</h2>
A random forest is an ensemble learning method that combines the predictions of multiple decision trees to improve accuracy and robustness. Each tree contributes a vote, and the final decision is made based on the majority vote from all the (decision) trees.

<h2>Decision Trees</h2>
A decision tree is a model used to make predictions by splitting data into branches based on feature values. Each branch represents a decision rule, leading to further splits until a final prediction is made at the leaf nodes. This method is intuitive and visually interpretable, making it useful for understanding complex decision-making processes.

<ul>
    <li>Root Node: The root node is the topmost node of a decision tree, representing the entire dataset and the starting point for splitting data.</li>
    <li>Splitting: Splitting is the process of dividing a node into two or more sub-nodes based on specific conditions.</li>
    <li>Decision Node: A decision node is an intermediate node where decisions are made based on feature values, leading to further splits.</li>
    <li>Leaf Node: A leaf node, or terminal node, is the endpoint of a branch representing the final output or decision.</li>
    <li>Parent/Child Node: A parent node splits into sub-nodes called child nodes, organizing the tree structure and defining decision paths.</li>
    <li>Pruning: Pruning involves removing unnecessary nodes to prevent overfitting and improve the model's generalization.</li>
    <li>Branching/Sub-Tree: Branching refers to dividing a node into sub-nodes, while a sub-tree is a smaller section of the decision tree starting from a particular node.</li>
</ul>



<h2>Why a Random Forest?</h2>
Even small changes to the input data can **dramatically** alter the overall structure of a decision tree, making it unstable. Decision trees are often relatively inaccurate, with many other predictors performing better on similar data. For data including categorical variables with different numbers of levels, information gain in decision trees is biased in favor of attributes with more levels. Calculations can become very complex, particularly if many values are uncertain and/or if many outcomes are linked. These are some of the reasons it's preferable to use Random Forest, as it helps overcome some of the weaknesses of decision trees. However, no model is perfect. 

<h2>Bagging</h2>
Bagging, short for bootstrap aggregating, is a technique used in random forests to improve the stability and accuracy of machine learning models. It involves generating multiple subsets of the original dataset through random sampling with replacement. Each subset is used to train a separate decision tree. The final prediction of the random forest is obtained by averaging the predictions (for regression) or taking a majority vote (for classification) from all the individual trees. This process reduces variance and helps prevent overfitting, leading to more robust and reliable predictions.

<h2>Supervised Learning</h2>
Random Forest uses supervised learning. In supervised learning, we provide the model with a "LABELED" data set which tells the model what the "correct" value it should be. Random Forest, is an example of a supervised learning algorithim because we provide the model a labeled data set.

<h2>Data Preprocessing</h2>
 This section of the code is responsible for data preprocessing.
 It includes steps to clean, transform, and prepare the data for further analysis.

In [ ]:
# import libraries
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report

from xgboost import XGBClassifier
import xgboost as xgb


<h3>Grabbing data from Yahoo Finance</h3>
Downloads daily price history for a list of tickers and saves it to a CSV file. You need to run this every day to get the newest price data.


In [ ]:
def grab_price_data():

    # Define the list of tickers
    tickers_list = ['JPM', 'COST', 'IBM', 'HD', 'ARWR'] #JPMorgan, Costco, IBM, Home Depot, Arrowhead Pharmaceuticals
    
    # I need to store multiple result sets.
    full_price_history = []

    for ticker in tickers_list:

        # Grab the daily price history for 2 years
        stock_data = yf.download(ticker, period='2y', interval='1d')

        # Reset index to make 'Date' a column instead of the index
        stock_data.reset_index(inplace=True)

        # Rename the 'Date' column to 'datetime' to match the expected column names
        stock_data.rename(columns={'Date': 'datetime', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'}, inplace=True)

        # Add a 'symbol' column to the dataframe
        stock_data['symbol'] = ticker

        # Append the dataframe to the list
        full_price_history.append(stock_data)

    # Concatenate all dataframes in the list into a single dataframe
    all_data = pd.concat(full_price_history)

    # Dump the data to a CSV file, don't have an index column
    all_data.to_csv('price_data.csv', index=False)

# Run the function to grab the data
grab_price_data()


In [ ]:
if os.path.exists('price_data.csv'):
    # Load the data
    price_data = pd.read_csv('price_data.csv')
else:
    # Grab the data and store it.
    grab_price_data()
    # Load the data
    price_data = pd.read_csv('price_data.csv')

# Display the head before moving on.
price_data.head()

<h2>Reformatting the Data</h2>
Process the price data by selecting specific columns, sorting the data, calculating change in price, and handling null values.

In [ ]:

price_data = price_data[["symbol", "datetime", "close", "high", "low", "open", "volume"]]

#Sort the data by symbol and datetime
price_data.sort_values(by=['symbol', 'datetime'], inplace=True) #inplace=true means that the changes are saved to the dataframe and we dont need another variable to store the changes

price_data["change_in_price"] = price_data["close"].diff()

#identify the rows where the ticker symbol changes, and then shift the change_in_price value to NaN
mask = price_data["symbol"] != price_data["symbol"].shift(1)

#change change_in_price to NaN where needed
price_data["change_in_price"] = np.where(mask == True, np.NaN, price_data["change_in_price"]) #this is a numpy function that takes 3 arguments, the first is the condition, the second is the value to be used if the
                                                                                              #condition is true, and the third is the value to be used if the condition is false
                                                                                              
#see all null vals                                                                                              
price_data[price_data.isna().any(axis=1)]


<h2> Calculating Indicators</h2>
RSI: The Relative Strength Index (RSI) is a momentum oscillator used in technical analysis to measure the speed and change of price movements. It ranges from 0 to 100 and is typically used to identify overbought or oversold conditions in a market, with values above 70 indicating overbought conditions and values below 30 indicating oversold conditions. RSI helps traders make decisions about potential price reversals or continuation trends.

RSI = 100 + (100/(1+RS) where RS = (Avg. Gain/Avg. Loss)


<h4>Indicator Code</h4>
A lot of the same steps are applied to calculate the indicator that we will be using. Generally, the code follows this format:

<ol>
    <li>Copy the desired columns and store them as variables.</li>
    <li>Group columns by symbol, select column we want to transform, and use the transform method anf a lambda function to calculate the indicator.</li>
    <li>Store values back in main data frame.</li>
</ol>

Note: Slight variations will exist, however these are negligible enough not to be mentioned here. 

In [ ]:
# Calculate the 14 day RSI
n = 14

# First make a copy of the data frame twice (exact same both times)
up_df, down_df = price_data[['symbol','change_in_price']].copy(), price_data[['symbol','change_in_price']].copy()

# For up days, if the change is less than 0 set to 0. This will only contain positive values.
up_df.loc['change_in_price'] = up_df.loc[(up_df['change_in_price'] < 0), 'change_in_price'] = 0

# For down days, if the change is greater than 0 set to 0. This will only contain negative values.
down_df.loc['change_in_price'] = down_df.loc[(down_df['change_in_price'] > 0), 'change_in_price'] = 0

# We need change in price to be absolute, so we use abs to make it so. (Only for down days)
down_df['change_in_price'] = down_df['change_in_price'].abs()

# Calculate the EWMA (Exponential Weighted Moving Average), meaning older values are given less weight compared to newer values.
ewma_up = up_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean()) #The ewm function is exponential weighted moving average over a specified span of n days
ewma_down = down_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean()) #The groupby makes it so the ewm is done by symbol and not over the whole dataset


# Calculate the Relative Strength
relative_strength = ewma_up / ewma_down

# Calculate the Relative Strength Index
relative_strength_index = 100.0 - (100.0 / (1.0 + relative_strength))

# Add the info to the data frame.
price_data['down_days'] = down_df['change_in_price']
price_data['up_days'] = up_df['change_in_price']
price_data['RSI'] = relative_strength_index

# Display the head.
price_data.head(10)

<h2>Stochastic Oscillator</h2>

The stochastic oscillator is a momentum indicator that compares a particular closing price of a security to a range of its prices over a specific period, typically 14 days. It helps identify overbought and oversold conditions by measuring the location of the closing price relative to the high-low range over that period.

%K=(Highest High − Lowest Low/Current Close − Lowest Low)×100

Again, the steps here are nearly identical.

In [ ]:
# Calculate the Stochastic Oscillator
n = 14

# Make a copy of the high and low column. (Instead of change in price like we did with RSI)
low_14, high_14 = price_data[['symbol','low']].copy(), price_data[['symbol','high']].copy()

# Group by symbol, then apply the rolling function and grab the Min and Max.
low_14 = low_14.groupby('symbol')['low'].transform(lambda x: x.rolling(window = n).min()) #Pandas dataframe.rolling() function provides the feature of rolling window calculations.
high_14 = high_14.groupby('symbol')['high'].transform(lambda x: x.rolling(window = n).max())

# Calculate the Stochastic Oscillator.
k_percent = 100 * ((price_data['close'] - low_14) / (high_14 - low_14))

# Add the info to the data frame.
price_data['low_14'] = low_14
price_data['high_14'] = high_14
price_data['k_percent'] = k_percent

# Display the head.
price_data.head(20)

<h2>Williams %R</h2>

Williams %R ranges from -100 to 0. When its value is above -20, it indicates a sell signal and when its value is below -80, it indicates a buy signal. It is nearly identical in formula to the Stochastic Oscillator. 

%R=( Highest High − Lowest Low/ Highest High − Current Close)×−100

Again, the steps here are nearly identical.

In [ ]:
# Calculate the Williams %R. Until r_percent is calculated, the code is IDENTICAL to the Stochastic Oscillator.
n = 14

# Make a copy of the high and low column.
low_14, high_14 = price_data[['symbol','low']].copy(), price_data[['symbol','high']].copy()

# Group by symbol, then apply the rolling function and grab the Min and Max.
low_14 = low_14.groupby('symbol')['low'].transform(lambda x: x.rolling(window = n).min())
high_14 = high_14.groupby('symbol')['high'].transform(lambda x: x.rolling(window = n).max())

# Calculate William %R indicator.
r_percent = ((high_14 - price_data['close']) / (high_14 - low_14)) * - 100

# Add the info to the data frame. 
price_data['r_percent'] = r_percent

# Display the head.
price_data.head(20)

<h2>Moving Average Convergence Divergence</h2>

The Moving Average Convergence Divergence (MACD) is a trend-following momentum indicator that shows the relationship between two moving averages of a security's price. It is used to identify potential buy and sell signals based on the convergence and divergence of these moving averages.

MACD Line:
MACD Line=EMA_12(c)−EMA_26(c) where C = closing price

Signal Line:
Signal Line=EMA_9(MACD)


Again, the steps here are nearly identical.

In [ ]:
# Calculate the MACD
ema_26 = price_data.groupby('symbol')['close'].transform(lambda x: x.ewm(span = 26).mean())
ema_12 = price_data.groupby('symbol')['close'].transform(lambda x: x.ewm(span = 12).mean())
macd = ema_12 - ema_26

# Calculate the EMA
ema_9_macd = macd.ewm(span = 9).mean()

# Store the data in the data frame.
price_data['MACD'] = macd
price_data['MACD_EMA'] = ema_9_macd

# Print the head.
price_data.head(20)

<h2>Price Rate of Change</h2>

The Price Rate of Change (ROC) is a momentum oscillator that measures the percentage change in price between the current price and the price a certain number of periods ago. It helps identify the strength of price movements and potential reversals.

ROC=(Price n-Periods Ago / Current Price − Price n-Periods Ago)x100

Again, the steps here are nearly identical.

In [ ]:
# Calculate the Price Rate of Change
n = 9

# Calculate the Rate of Change in the Price, and store it in the Data Frame.
price_data['Price_Rate_Of_Change'] = price_data.groupby('symbol')['close'].transform(lambda x: x.pct_change(periods = n)) #pct_change computes the fractional change from the immediately previous row by default. This is useful in comparing the fraction of change in a time series of elements.

# Print the first 30 rows
price_data.head(20)

<h2>On Balance Volume</h2>

Although similar, this will be harder to calculate. 

The On-Balance Volume (OBV) is a cumulative momentum indicator that measures buying and selling pressure by adding the volume on up days and subtracting the volume on down days. It helps confirm trends and potential reversal points in a security's price.

Formula (I dont know how do do piecewise in markdown):

def calculate_obv(close_prices, volumes):
    obv = [0]  # Start with an initial OBV value of 0
    
    for i in range(1, len(close_prices)):
        if close_prices[i] > close_prices[i - 1]:
            obv.append(obv[-1] + volumes[i])
        elif close_prices[i] < close_prices[i - 1]:
            obv.append(obv[-1] - volumes[i])
        else:
            obv.append(obv[-1])
    
    return obv

This portion involves using the apply method to calculate the On Balance Volume by applying a custom function to each row. The function calculates the difference in closing prices and adjusts the volume: adding if the price increased, subtracting if it decreased, and leaving it unchanged if the price remained the same.


In [ ]:
def obv(group):
    """
    Calculate the On Balance Volume (OBV) for a given group of data.

    Parameters:
    - group (pandas.DataFrame): A pandas DataFrame containing the 'volume' and 'close' columns.

    Returns:
    - pandas.Series: A pandas Series containing the calculated OBV values.

    Example:
    >>> data = pd.DataFrame({'volume': [100, 200, 150, 300], 'close': [10, 12, 11, 13]})
    >>> obv(data)
    0    100
    1    300
    2    150
    3    450
    dtype: int64
    """
    # Grab the volume and close column.
    volume = group['volume']
    change = group['close'].diff()

    # intialize the previous OBV
    prev_obv = 0
    obv_values = []

    # calculate the On Balance Volume
    for i, j in zip(change, volume):

        if i > 0:
            current_obv = prev_obv + j
        elif i < 0:
            current_obv = prev_obv - j
        else:
            current_obv = prev_obv

        # OBV.append(current_OBV)
        prev_obv = current_obv
        obv_values.append(current_obv)
    
    # Return a panda series.
    return pd.Series(obv_values, index = group.index)
        

# apply the function to each group
obv_groups = price_data.groupby('symbol').apply(obv)

# add to the data frame, but drop the old index, before adding it.
price_data['On Balance Volume'] = obv_groups.reset_index(level=0, drop=True)

# display the data frame.
price_data.head(20)

<h2>Creating the Prediction Column</h2>

Now that we have our technical indicators calculated and our price data cleaned up, we are almost ready to build our model. However, we are missing one critical piece of information: the column we wish to predict. Our goal is to predict whether the next day will be a down day or an up day, making this a classification problem with two discrete groups.

To create our prediction column, we will group our DataFrame by each symbol and select the close column to determine if the stock closed up or down for any given day. We will use the transform method to apply a lambda function that uses the shift() function to return True on up days and False on down days. We then multiply those results by 1 so that all False turn into 0s and all True turn into 1s. We then add this to our Prediction Column.

In [ ]:
closed_groups = price_data.groupby('symbol')['close']

closed_groups = closed_groups.transform(lambda x: x.shift(1) < x)

price_data["Prediction"] = closed_groups * 1

<h2>Removing NaN Values</h2>

The random forest can't accept NaN values, so we will need to remove them before feeding the data in. The code below prints the number of rows before dropping the NaN values, use the dropna method to remove any rows NaN values and then displays the number of rows after dropping the NaN values.

In [ ]:
# We need to remove all rows that have an NaN value.
print('Before NaN Drop we have {} rows and {} columns'.format(price_data.shape[0], price_data.shape[1]))

# Any row that has a `NaN` value will be dropped.
price_data = price_data.dropna()

# Display how much we have left now.
print('After NaN Drop we have {} rows and {} columns'.format(price_data.shape[0], price_data.shape[1]))

# Print the head.
price_data.head()

<h2>Selecting Indicators to Use in our Model</h2>

We need to identify our input columns which are the following:
<ul>
    <li>RSI</li>
    <li>Stochastic Oscillator</li>
    <li>William %R</li>
    <li>Price Rate of Change</li>
    <li>MACD</li>
    <li>On Balance Volume</li>
</ul>

These will serve as our x, and our Y column will be the prediction column.

Once we've selected our columns, we need to split the data into a training and test set. SciKit learn makes this easy by providing the train_test_split object, which will take our X_Cols and Y_Cols and split them based on the size we input. In our case, let's have the test_size be '20 %. For reproducibility, the train_test_splitobject provides therandom_state` argument that will split the data along the same dimensions every time.

In [ ]:
# Grab our X & Y Columns.
X_Cols = price_data[['RSI','k_percent','r_percent','Price_Rate_Of_Change','MACD','On Balance Volume']]
Y_Cols = price_data['Prediction']

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X_Cols, Y_Cols, random_state = 0)

# Create a Random Forest Classifier (using xgb boost increased success rate by 2%)
rand_frst_clf = XGBClassifier(n_estimators=10, max_depth=2, learning_rate=1, objective='binary:logistic') 

# Fit the data to the model
rand_frst_clf.fit(X_train, y_train)

# Make predictions
y_pred = rand_frst_clf.predict(X_test)

<h2>Accuracy</h2>

SciKit learn makes the process of evaluating our model very easy by providing a bunch of built-in metrics that we can call.

One of those metrics is the accuracy_score.

The accuracy_score function computes the accuracy, either the fraction (default) or the count (normalize=False) of correct predictions. Accuracy is defined as the number of accurate predictions the model made on the test set. Imagine we had three TRUE values [1, 2, 3], and our model predicted the following values [1, 2, 4] we would say the accuracy of our model is 66 %.

In [ ]:
print('Correct Prediction (%): ', accuracy_score(y_test, rand_frst_clf.predict(X_test), normalize = True) * 100.0)

<h2>Classification Report</h2>

Accuracy:
Accuracy measures the portion of all testing samples classified correctly and is defined as the following:
\[ \text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}} \]

Recall:
Recall (also known as sensitivity) measures the ability of a classifier to correctly identify positive labels and is defined as the following:
\[ \text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}} \]
The recall is intuitively the ability of the classifier to find all the positive samples. The best value is 1, and the worst value is 0.

Specificity:
Specificity measures the classifier’s ability to correctly identify negative labels and is defined as the following:
\[ \text{Specificity} = \frac{\text{TN}}{\text{TN} + \text{FP}} \]

Precision:
Precision measures the proportion of all correctly identified samples in a population of samples which are classified as positive labels and is defined as the following:
\[ \text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}} \]
The precision is intuitively the ability of the classifier not to label as positive a sample that is negative. The best value is 1, and the worst value is 0.

Interpreting the Classification Report:
When it comes to evaluating the model, we generally look at the accuracy. If our accuracy is high, it means our model is correctly classifying items. In some cases, we will have models that may have low precision or high recall. It's difficult to compare two models with low precision and high recall or vice versa. To make results comparable, we use a metric called the F-Score. The F-score helps to measure Recall and Precision at the same time. It uses Harmonic Mean in place of Arithmetic Mean by punishing the extreme values more.



In [ ]:
# Define the traget names
target_names = ['Down Day', 'Up Day']

# Build a classifcation report
report = classification_report(y_true = y_test, y_pred = y_pred, target_names = target_names, output_dict = True)

# Add it to a data frame, transpose it for readability. (transpose is a pandas function that swaps the rows and columns)
report_df = pd.DataFrame(report).transpose()

report_df

<h2>Feature Importance</h2>

Feature importance is a technique used in machine learning to determine the relative significance of each feature in a predictive model. It helps in identifying which features contribute the most to the model’s predictions, allowing for better model interpretation and potential feature selection. In tree-based models like Random Forests, feature importance is typically calculated by measuring the decrease in node impurity (e.g., Gini impurity or Accuracy) for each feature across all trees in the model. Features with higher importance scores are more influential in predicting the target variable.



<h4>Calculating the Feature Importance</h4>

Like all the previous steps, SkLearn makes this process very easy. Take your rand_frst_clf and call the feature_importances_ property. This will return all of our features and their importance measurement. Store the values in a Pandas.Series object and sore the values.

Feature importance can be calculated two ways in Random Forest:

Gini-Based Importance
Accuracy-Based Importance

Here is how both measures of importance are calculated.

With sklearn they use the Gini-Importance metric for the Random Forest Algorithm.

We can see in our model, that the most important feature is k_percent and our least important feature is OBV.

In [ ]:
# Calculate feature importance and store in pandas series
feature_imp = pd.Series(rand_frst_clf.feature_importances_, index=X_Cols.columns).sort_values(ascending=False)
feature_imp

<h2>ROC Curve</h2>

The greater the area under the curve, the better.

In [ ]:
rfc_disp = RocCurveDisplay.from_estimator(rand_frst_clf, X_test, y_test)
rfc_disp.plot()

<h2>Improvements</h2>

To determine the optimal number of estimators for a Random Forest, you need to try different values rather than relying on a fixed number. This can be achieved using the RandomizedSearchCV method from sklearn, which tests a wide range of possible values for each hyperparameter using cross-validation to find the best combination. 

In [ ]:
param_grid = {
    'n_estimators': [10, 100, 200],
    'max_depth': [1,2,3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2,0.3,0.5,0.7,0.8,1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [1, 1.5, 2]
}
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')




<h2>Running Randomized Search</h2>

In [ ]:
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_grid, n_iter=50, scoring='accuracy', cv=3, verbose=1, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)
print("Best parameters found: ", random_search.best_params_)
print("Best accuracy score: ", random_search.best_score_)

<h2>Testing Improved RF</h2>

Print new tests then print optimal best estimations. 

In [ ]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy: ", accuracy)
